In [183]:
import pandas as pd
import numpy as np

- Summarize et 
- Bağlaç listesi bul/oluştur.
- Summarize edilmiş place'lerin cümlelerini birleştir.
- Her summary bitişi bağlaçları random yerleştir.
- Her gün oluşan textten google t5-base ile title generate et

In [184]:
data_df = pd.read_excel("whole_data_cleaned.xlsx")
data_df

,name,rating,tags,website,place_links,territory_id,description,locationYX
0,Kadikoy Ferry Terminal,4.6,"1,9,14",https://thetravelauthor.com/kadikoy-ferry-ista...,https://www.google.com/maps/place/Kadikoy+Ferr...,2,The Kadıköy Ferry Terminal serves as a major t...,"40.99269778351916, 29.023280555674663"
1,Kadikoy Bull Statue,4.5,"1,3,6",https://tr.wikipedia.org/wiki/Kad%C4%B1k%C3%B6...,https://www.google.com/maps/place/Kad%C4%B1k%C...,2,The Kadıköy Bull Statue is a beloved symbol of...,"40.990473264783475, 29.029131932189433"
2,Kadikoy-moda Streets,4.6,"13,14,15",https://tr.wikipedia.org/wiki/Kad%C4%B1k%C3%B6y,https://www.google.com/maps/place/Kad%C4%B1k%C...,2,The Kadıköy Marketplace is a vibrant hub of ac...,"41.0300084184215, 28.98441527977153"
3,Moda Beach Park,4.7,"9,11,12","https://tr.wikipedia.org/wiki/Moda,_Kad%C4%B1k...",https://www.google.com/maps/place/Moda+Sahil+P...,2,Moda Beach is a serene haven nestled along the...,"40.98000940235465, 29.026848556608424"
4,IDEA Kadikoy,4.6,"9,14,16",https://en.ideakadikoy.org/,https://www.google.com/maps/place/%C4%B0DEA+Ka...,2,"IDEA Kadıköy, developed by Kadıköy Municipalit...","40.98015752580784, 29.02810950338273"
...,...,...,...,...,...,...,...,...
183,Palace of the Porphyrogenitus,4.5,"2,3,4,7",https://en.wikipedia.org/wiki/Palace_of_the_Po...,"https://www.google.com/maps/dir//Ayvansaray,+3...",3,The Palace of the Porphyrogenitus is a remarka...,"41.03788124723142, 28.93945445055248"
184,Column of Marcian,4.3,"1,2,3,7",https://en.wikipedia.org/wiki/Column_of_Marcian,"https://www.google.com/maps/dir/41.1009024,28....",3,The Column of Marcian stands as a magnificent ...,"41.02630809319681, 28.95342213409043"
185,Beyazıt Tower,4.5,"1,2,3,7",https://en.wikipedia.org/wiki/Beyaz%C4%B1t_Tower,https://www.google.com/maps/dir//Beyaz%C4%B1t+...,3,The Beyazıt Tower stands proudly in Istanbul's...,"41.01817417223928, 28.96347312671263"
186,Grand Bazaar,4.1,"1,2,3,7,9,17","https://en.wikipedia.org/wiki/Grand_Bazaar,_Is...","https://www.google.com/maps/dir//Beyaz%C4%B1t,...",3,The Grand Bazaar is a bustling labyrinth of na...,"41.014800403046365, 28.967378072061717"


In [185]:
data_df.drop(["rating","tags","website","place_links","territory_id","locationYX"], axis=1, inplace=True)
data_df

,name,description
0,Kadikoy Ferry Terminal,The Kadıköy Ferry Terminal serves as a major t...
1,Kadikoy Bull Statue,The Kadıköy Bull Statue is a beloved symbol of...
2,Kadikoy-moda Streets,The Kadıköy Marketplace is a vibrant hub of ac...
3,Moda Beach Park,Moda Beach is a serene haven nestled along the...
4,IDEA Kadikoy,"IDEA Kadıköy, developed by Kadıköy Municipalit..."
...,...,...
183,Palace of the Porphyrogenitus,The Palace of the Porphyrogenitus is a remarka...
184,Column of Marcian,The Column of Marcian stands as a magnificent ...
185,Beyazıt Tower,The Beyazıt Tower stands proudly in Istanbul's...
186,Grand Bazaar,The Grand Bazaar is a bustling labyrinth of na...


In [186]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import random

# Initialize the tokenizer and model
model_name = 't5-base'  # or 't5-large' if you prefer
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to generate summaries
def generate_summary(description):
    input_ids = tokenizer.encode("summarize: " + description, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(input_ids, max_length=50, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Generate summaries for each description
tqdm.pandas()
data_df['summary'] = data_df['description'].progress_apply(generate_summary)
data_df

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 188/188 [14:35<00:00,  4.66s/it]


,name,description,summary
0,Kadikoy Ferry Terminal,The Kadıköy Ferry Terminal serves as a major t...,the Kadköy Ferry Terminal serves as a major tr...
1,Kadikoy Bull Statue,The Kadıköy Bull Statue is a beloved symbol of...,the Kadköy Bull Statue is a beloved symbol of ...
2,Kadikoy-moda Streets,The Kadıköy Marketplace is a vibrant hub of ac...,the Kadköy Marketplace offers a sensory feast ...
3,Moda Beach Park,Moda Beach is a serene haven nestled along the...,"with its gentle waves and clear blue waters, M..."
4,IDEA Kadikoy,"IDEA Kadıköy, developed by Kadıköy Municipalit...",IDEA Kadköy is a dynamic public space fosterin...
...,...,...,...
183,Palace of the Porphyrogenitus,The Palace of the Porphyrogenitus is a remarka...,the palace of the Porphyrogenitus dates back t...
184,Column of Marcian,The Column of Marcian stands as a magnificent ...,the Column of Marcian stands as a magnificent ...
185,Beyazıt Tower,The Beyazıt Tower stands proudly in Istanbul's...,built in the early 19th century during the Ott...
186,Grand Bazaar,The Grand Bazaar is a bustling labyrinth of na...,the grand bazaar is a bustling labyrinth of na...


In [213]:
# Define the number of days
num_days = 2  # Change this to your desired number of days

# Calculate the number of places
num_places = len(data_df)

# Create a list of days
days = np.arange(1, num_days + 1)

# Randomly assign each place to one of the days
data_df['day'] = np.random.choice(days, size=num_places, replace=True)

# Ensure that each day has exactly 5 places
places_per_day = num_places // num_days
day_assignment = np.repeat(days, places_per_day)

# Distribute the remaining places randomly to days
remaining_places = num_places % num_days
if remaining_places > 0:
    extra_places_day = np.random.choice(days, size=remaining_places, replace=False)
    day_assignment = np.concatenate((day_assignment, extra_places_day))

# Shuffle the day assignment
np.random.shuffle(day_assignment)
data_df['day'] = day_assignment

# Filter places so that each day has exactly 5 places
filtered_data = pd.concat([data_df[data_df['day'] == day].sample(n=5, replace=False) for day in days])

# Display the filtered DataFrame
filtered_data

,name,description,summary,day
172,Sirkeci Railway Station,"Sirkeci Railway Station, situated in Istanbul,...","Sirkeci Railway Station, situated in Istanbul,...",1
58,Historical Plane Tree of Çengelköy,The historical plane tree in Çengelköy stands ...,the plane tree in engelköy stands as a symbol ...,1
176,The Stone of Million,The Stone of Million is a significant landmark...,the Stone of Million is situated in the histor...,1
43,Istanbul Toy Musesum,The Istanbul Toy Museum is a delightful cultur...,the Istanbul Toy Museum showcases a vast colle...,1
17,Moda Sea Club,Moda Sea Club is a popular seaside venue in Ka...,moda sea club is a popular seaside venue in Ka...,1
54,Hababam Class Museum,"The Hababam Class Museum, nestled in Istanbul'...",the Hababam Class Museum offers a nostalgic jo...,2
0,Kadikoy Ferry Terminal,The Kadıköy Ferry Terminal serves as a major t...,the Kadköy Ferry Terminal serves as a major tr...,2
169,Cistern of Theodosius,"The Cistern of Theodosius, situated beneath th...",the cistern was built during the reign of Empe...,2
76,Aşiyan Museum,Aşiyan Museum is the house where poet Tevfik F...,the house where poet Tevfik Fikret lived has b...,2
28,Hasanpasa Mosque,Hasanpaşa Mosque is a historic mosque located ...,"the mosque features elegant domes, slender min...",2


In [214]:
import spacy
import random
import nltk
from nltk.tokenize import sent_tokenize

# Load the English model
nlp = spacy.load("en_core_web_sm")

def get_sentence_starters():
    return [
        "Furthermore,",
        "Additionally,",
        "In addition,",
        "On top of that,",
        "To add to that,",
        "Next,",
        "Afterwards"
    ]

def improve_fluency(summaries):
    processed_summaries = []
    for i, summary in enumerate(summaries):
        sentences = sent_tokenize(summary)
        for j, sentence in enumerate(sentences):
            sentence = sentence.capitalize()
            if j == 0 and i > 0:  # Add a transition at the start of a new summary
                transition = random.choice(get_sentence_starters())
                sentence = f"{transition} {sentence}"
            processed_summaries.append(sentence)
    return ''.join(processed_summaries)

# Function to combine summaries with conjunctions
def combine_summaries(group):
    combined_summary = ""
    for i, summary in enumerate(group['summary']):
        combined_summary += summary
        if i < len(group['summary']) - 1:
            combined_summary += " " + random.choice(get_sentence_starters()) + " "
    return combined_summary

# Combine summaries group by day
final_summaries = filtered_data.groupby('day').apply(combine_summaries).reset_index()
final_summaries.columns = ['day', 'combined_summary']

# Display the final summaries
print(final_summaries)


   day                                   combined_summary
0    1  Sirkeci Railway Station, situated in Istanbul,...
1    2  the Hababam Class Museum offers a nostalgic jo...


In [215]:
final_summaries["combined_summary"][1]

"the Hababam Class Museum offers a nostalgic journey into a significant era of Turkish cinema and popular culture. visitors can marvel at silicone sculptures of the film's actors, peruse posters and postcards showcasing memorable On top of that, the Kadköy Ferry Terminal serves as a major transportation hub connecting the Asian side of Istanbul to the European side via ferry services. Located in the heart of Kadköy, the terminal is surrounded by Additionally, the cistern was built during the reign of Emperor Theodosius II in the 5th century AD. it is also known as the Basilica Cistern due to its location beneath a former basilica Next, the house where poet Tevfik Fikret lived has been serving as a museum since 1945. the belongings of Tevfik Fikret and his family are exhibited. Afterwards the mosque features elegant domes, slender minarets, and intricate decorations. visitors can admire the mosque's exterior facade and tranquil courtyard. the mosque serves as a place of worship and comm

In [219]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the BART tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("AryanLala/autonlp-Scientific_Title_Generator-34558227")
model = AutoModelForSeq2SeqLM.from_pretrained("AryanLala/autonlp-Scientific_Title_Generator-34558227")

# Define a function to generate a title from the combined summary using the BART model
def generate_title_bart(combined_summary):
    input_text = f"First Day: {combined_summary}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(input_ids, max_length=32, num_beams=4, early_stopping=True)
    title = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return title

# Define a function to generate the final output string with titles
def generate_final_output_with_titles(final_summaries):
    day_strings = []
    for index, row in final_summaries.iterrows():
        day_number = row['day']
        combined_summary = row['combined_summary']
        title = generate_title_bart(combined_summary)
        day_string = f"Day {day_number}: {title}\n{combined_summary}\n"
        day_strings.append(day_string)
    return ''.join(day_strings)

# Generate the final output string with titles
final_output = generate_final_output_with_titles(final_summaries)

# Display the final output
print(final_output)


Day 1: On the Road to Istanbul
Sirkeci Railway Station, situated in Istanbul, is a renowned transportation landmark. the station is the terminus of the legendary Orient Express, which linked Istanbul to Paris. the station's central location near major tourist Next, the plane tree in engelköy stands as a symbol of endurance and timelessness. its gnarled branches stretch out over the bustling streets below. its weathered bark tells stories of bygone Afterwards the Stone of Million is situated in the historic Sultanahmet square, at the heart of Istanbul's old city. the stone served as the starting point of the road network in the Byzantine Empire. it is considered one On top of that, the Istanbul Toy Museum showcases a vast collection of antique toys dating back to the 18th century. interactive workshops and educational programs offer hands-on experiences for children and families. Next, moda sea club is a popular seaside venue in Kadköy. guests can relax by the pool, soak up the sun on t

In [ ]:
import pandas as pd
import numpy as np
import spacy
import random
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load SpaCy and NLTK data (download if necessary)
nltk.download('punkt')

# Load models
nlp = spacy.load("en_core_web_sm")
tokenizer = AutoTokenizer.from_pretrained("AryanLala/autonlp-Scientific_Title_Generator-34558227")
model = AutoModelForSeq2SeqLM.from_pretrained("AryanLala/autonlp-Scientific_Title_Generator-34558227")


# --- Helper Functions ---

def get_sentence_starters():
    return ["Furthermore,", "Additionally,", "In addition,", "On top of that,", "To add to that,", "Next,", "Afterwards"]


def improve_fluency(summaries):
    processed_summaries = []
    for i, summary in enumerate(summaries):
        sentences = sent_tokenize(summary)
        for j, sentence in enumerate(sentences):
            sentence = sentence.capitalize()
            if j == 0 and i > 0:  # Add a transition at the start of a new summary
                transition = random.choice(get_sentence_starters())
                sentence = f"{transition} {sentence}"
            processed_summaries.append(sentence)
    return ''.join(processed_summaries)


def combine_summaries(group):
    combined_summary = improve_fluency(group['summary'])
    return combined_summary


def generate_title_bart(combined_summary):
    input_text = f"First Day: {combined_summary}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(input_ids, max_length=64, num_beams=4, early_stopping=True)
    title = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return title


def generate_final_output_with_titles(final_summaries):
    day_strings = []
    for index, row in final_summaries.iterrows():
        day_number = row['day']
        combined_summary = row['combined_summary']
        title = generate_title_bart(combined_summary)
        day_string = f"Day {day_number}: {title}\n{combined_summary}\n"
        day_strings.append(day_string)
    return ''.join(day_strings)


# --- Main Function ---
def generate_travel_plan(data_df, num_days=2):
    # Assign Places to Days
    num_places = len(data_df)
    days = np.arange(1, num_days + 1)

    data_df['day'] = np.random.choice(days, size=num_places, replace=True)
    places_per_day = num_places // num_days
    day_assignment = np.repeat(days, places_per_day)

    remaining_places = num_places % num_days
    if remaining_places > 0:
        extra_places_day = np.random.choice(days, size=remaining_places, replace=False)
        day_assignment = np.concatenate((day_assignment, extra_places_day))

    np.random.shuffle(day_assignment)
    data_df['day'] = day_assignment

    filtered_data = pd.concat([data_df[data_df['day'] == day].sample(n=5, replace=False) for day in days])

    # Combine and Improve Summaries
    final_summaries = filtered_data.groupby('day').apply(combine_summaries).reset_index()
    final_summaries.columns = ['day', 'combined_summary']

    # Generate Titles and Final Output
    final_output = generate_final_output_with_titles(final_summaries)
    return final_output



if __name__ == "__main__":
    # --- Example Usage (for testing in Python) ---
    data_df = pd.read_csv("summarized_data.csv") 
    travel_plan = generate_travel_plan(data_df, num_days=2)
    print(travel_plan)
